In [52]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [53]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

spectrum_train_scaled = spectrum_train.multiply(10**12)
spectrum_test_scaled = spectrum_test.multiply(10**12)

In [54]:
temp_to_spec = False

if temp_to_spec:
    X_train = temp_train
    X_test = temp_test
    y_train = spectrum_train_scaled
    y_test = spectrum_test_scaled
else:
    X_train = spectrum_train_scaled
    X_test = spectrum_test_scaled
    y_train = temp_train
    y_test = temp_test

In [55]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [56]:
input_data = df_to_tensor(X_train)
output_data = df_to_tensor(y_train)

In [57]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [58]:
input_size = len(X_train.columns)
output_size = len(y_train.columns)
model = Net(input_size=input_size, output_size=output_size)

In [59]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Rprop(model.parameters(), lr=0.001) 

In [60]:
num_epochs = 15
batch_size = 5
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 1/15, Loss: 740.910400
Epoch 2/15, Loss: 145.343353
Epoch 3/15, Loss: 360.804718
Epoch 4/15, Loss: 977.335754
Epoch 5/15, Loss: 583.471252
Epoch 6/15, Loss: 416.440552
Epoch 7/15, Loss: 873.448853
Epoch 8/15, Loss: 736.180847
Epoch 9/15, Loss: 5085.476074
Epoch 10/15, Loss: 318.386444
Epoch 11/15, Loss: 456.125885
Epoch 12/15, Loss: 1629.762085
Epoch 13/15, Loss: 565.136230
Epoch 14/15, Loss: 622.961487
Epoch 15/15, Loss: 830.222290


In [61]:
predictions = model(df_to_tensor(X_train))

In [62]:
pd.DataFrame(predictions.detach().numpy(), columns=y_train.columns).head()

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,layer 11
0,161.080338,233.246338,58.641842,62.165504,70.652672,80.997620,102.720581,104.604683,168.697617,18.567511,39.641056
1,162.690231,234.170654,57.657574,63.536243,71.684181,80.089645,100.408981,106.465378,170.684494,20.104809,39.055023
2,161.080338,233.246338,58.641842,62.165504,70.652672,80.997620,102.720581,104.604683,168.697617,18.567511,39.641056
3,168.915695,238.276535,55.364059,68.193214,75.632393,77.626045,93.434685,112.615486,177.611450,25.017790,37.415337
4,158.127655,231.606873,55.869408,61.167725,68.499092,80.585831,103.035126,105.200127,168.508240,18.789051,39.485027


In [63]:
y_test.head()

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,layer 11
0,410.761201,412.678794,409.482872,401.948393,398.928932,397.948393,401.340736,408.678794,416.761201,421.621765,421.254224
1,408.690133,411.339048,412.411804,410.608647,411.000000,407.948393,403.411804,400.018540,399.690133,402.961511,409.183156
2,414.296735,417.678794,413.018406,401.948393,395.393398,392.948393,397.805202,408.678794,420.296735,426.621765,424.789758
3,411.190133,415.669175,417.411804,414.938774,413.500000,407.948393,400.911804,395.688413,394.690133,398.631384,406.683156
4,415.482872,416.788008,412.019461,402.726045,398.928932,398.726045,403.877325,412.788008,421.482872,425.993518,424.749862
